In [1]:
import json
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.express as px
import gradio as gr
import base64
import io
import openai
import sqlite3
import pytz
from pathlib import Path
from tensorflow.keras.applications import efficientnet
from openai import OpenAI
from datetime import datetime, timedelta

2025-10-09 18:32:54.067861: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-09 18:32:54.159895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760056374.200640    5182 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760056374.212537    5182 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760056374.288668    5182 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
model = tf.keras.models.load_model('Models/Model(RGB)_v2.5.keras')

I0000 00:00:1760056378.955558    5182 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6122 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
api_key = Path('APi_Keys/API_Key_Open_AI.txt').read_text(encoding='utf-8').strip()

In [4]:
client = OpenAI(api_key=api_key)

### Funciones SQL

In [5]:
def init_db():
    conn = sqlite3.connect("Nutrivision_log.db")
    c = conn.cursor()
    c.execute('''
    CREATE TABLE IF NOT EXISTS registros (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT DEFAULT CURRENT_TIMESTAMP,
    carbs REAL,
    protein REAL,
    fat REAL,
    calories REAL,
    calories_from_macros REAL
    )
    ''')
    conn.commit()
    conn.close()

In [6]:
def load_records(db_path="Nutrivision_log.db"):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT * FROM registros ORDER BY timestamp DESC", conn)
    conn.close()
    return df

In [7]:
def delete_record_by_id(record_id, db_path="Nutrivision_log.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM records WHERE id = ?", (record_id,))
    conn.commit()
    conn.close()


### Funciones Gradio

In [8]:
with open('Models/normalization_stats.json') as f:
    stats = json.load(f)

In [9]:
def openai_is_food(image_bytes: bytes) -> bool:
    """
    Llama a la API de OpenAI (modelo vision/multimodal) para decidir si la imagen contiene comida.
    Retorna True si “es comida”, False de lo contrario.
    """
    # Codificamos la imagen en base64
    b64 = base64.b64encode(image_bytes).decode("utf-8")
    img_data = f"data:image/jpeg;base64,{b64}"

    # Construimos el mensaje para el modelo con visión (modelo que acepte imagen + texto)
    messages = [
        {
            "role": "system",
            "content": (
                "Eres un modelo que debe decidir si una imagen contiene un plato de comida. "
                "Responde con JSON del tipo {\"is_food\": true} o {\"is_food\": false}."
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "¿La siguiente imagen muestra comida comestible?"},
                {"type": "image_url", "image_url": {"url": img_data}}
            ]
        }
    ]
    resp = client.chat.completions.create(
        model="gpt-4o-2024-08-06",  # o el modelo vision que uses
        messages=messages,
        temperature=0.0,
        max_tokens=20
    )
    respuesta = resp.choices[0].message.content
    # Esperamos algo como {"is_food": true} o false
    try:
        obj = json.loads(respuesta)
        return bool(obj.get("is_food", False))
    except json.JSONDecodeError as e:
        print("Error parseando respuesta OpenAI (JSON):", respuesta, e)
    # Opcional: intentar fallback con eval si JSON no es válido
        try:
            obj2 = eval(respuesta)
            return bool(obj2.get("is_food", False))
        except Exception as e2:
            print("Error fallback parseando con eval:", respuesta, e2)
            return False


In [10]:
def predict_nutrition(image):
    img = tf.convert_to_tensor(np.array(image))  
    img = tf.image.resize(img, [300, 300])
    img = efficientnet.preprocess_input(img)
    img = tf.expand_dims(img, 0)
    
    preds = model.predict(img, verbose=0)
    macros_norm, cal_norm = preds
    

    carb_log = macros_norm[0, 0] * stats['carb_std'] + stats['carb_mean']
    prot_log = macros_norm[0, 1] * stats['prot_std'] + stats['prot_mean']
    fat_log = macros_norm[0, 2] * stats['fat_std'] + stats['fat_mean']
    cal_log = cal_norm[0, 0] * stats['cal_std'] + stats['cal_mean']

    carb = max(0, np.expm1(carb_log))
    prot = max(0, np.expm1(prot_log))
    fat = max(0, np.expm1(fat_log))
    cal = max(0, np.expm1(cal_log))
    
    # Calcular calorías desde macros
    cal_from_macros = 4*carb + 4*prot + 9*fat

    return {
        'carbs' : float(carb),
        'protein' : float(prot),
        'fat' : float(fat),
        'calories' : float(cal),
        'calories_from_macros' : float(cal_from_macros)
    }

In [11]:
def limpiar():
    gr.Info("Interfaz Limpiada.")
    return None, None, None, None, None, None, False

In [12]:
def recomendacion_LLM(nutrition : dict):
    carbs = nutrition.get("carbs", 0)
    prot = nutrition.get("protein", 0)
    fat = nutrition.get("fat", 0)
    cal = nutrition.get("calories", 0)
    cal_macros = nutrition.get("calories_from_macros", 0)
    prompt = (
        f"- Actua com un experto en en nutricion y responde: Tengo un platillo con los siguientes valores nutricionales estimados ->\n"
        f"- Carbohidratos: {carbs:.1f} g\n"
        f"- Proteína: {prot:.1f} g\n"
        f"- Grasa: {fat:.1f} g\n"
        f"- Calorías (modelo): {cal:.0f} kcal\n"
        f"- Calorías calculadas desde macros: {cal_macros:.0f} kcal\n\n"
        f"- Quiero que me des una recomendacion con respecto a estos macros tomando en cuenta 1. Objetivos Alimenticios [saciedad, para que objetivo es buena esta comida (Volumen, Definicion, Recomposicion, Mantenerse)]. 2 Objetivos Deportivos [Es buena o no para un dia de ejercicio, antes o despues de ejercicio]. Pero siempre haciendo enfasis en que es solo una sugerencia. \n"
        f"- Solo Devuelve tus respitesta en formato de 1. Objetivos Alimenticios, 2 Objetivos Deportivos, los valores de macros y calorias no los despliegues, puedes hacer referencia a ellos pero no los despliegues"
        f"- Toma en cuenta decir esta infromacion en menos de 400 tokens"    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Eres un nutricionista experto que da consejos personalizados."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

In [13]:
def nutrition_summary(nutrition_json: dict) -> str:
    """
    Convierte un diccionario con predicciones nutricionales en texto Markdown formateado.
    """
    cal_from_macros = nutrition_json['calories_from_macros']
    cal = nutrition_json['calories']
    
    return (
        f"### 📊 Valores Nutricionales del Platillo\n"
        f"- **Carbohidratos**: {nutrition_json['carbs']:.1f} g\n"
        f"- **Proteína**: {nutrition_json['protein']:.1f} g\n"
        f"- **Grasa**: {nutrition_json['fat']:.1f} g\n"
        f"- **Calorías**: {max(cal_from_macros,cal):.0f} - {min(cal_from_macros,cal):.0f} kcal"
    )


In [14]:
def macros_barplot(nutrition_json):
    macro_color_map = {
        'Carbohidratos': '#FF9999',
        'Proteína': '#99CC99', 
        'Grasa': '#9999FF'
    }
    
    df = pd.DataFrame({
        "Macronutriente": ["Carbohidratos", "Proteína", "Grasa"],
        "Gramos": [
            nutrition_json["carbs"],
            nutrition_json["protein"],
            nutrition_json["fat"]
        ]
    })

    fig = px.bar(
        df,
        x="Macronutriente",
        y="Gramos",
        color="Macronutriente",
        color_discrete_map=macro_color_map,
        title="Distribución de Macronutrientes"
    )

    fig.update_layout(
        yaxis_title="Cantidad (g)",
        xaxis_title=None,
        showlegend=False,
        template="none",  
        paper_bgcolor="#1E1E2E",  
        plot_bgcolor="#2E2E3E",   
        font_color="#E0E0E0"
    )

    return fig


In [15]:
def calorie_pie(nutrition_json):
    macro_color_map = {
        'Carbohidratos': '#FF9999',
        'Proteína': '#99CC99', 
        'Grasa': '#9999FF'
    }
    carb_kcal = nutrition_json["carbs"] * 4
    prot_kcal = nutrition_json["protein"] * 4
    fat_kcal = nutrition_json["fat"] * 9

    df = pd.DataFrame({
        "Macronutriente": ["Carbohidratos", "Proteína", "Grasa"],
        "Calorías": [carb_kcal, prot_kcal, fat_kcal]
    })

    fig = px.pie(
        df,
        names="Macronutriente",
        values="Calorías",
        title="Distribución Calórica por Macronutriente",
        color = 'Macronutriente',
        color_discrete_map=macro_color_map,
        hole=0.3 
    )

    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(template="none",  
                        paper_bgcolor="#1E1E2E",  
                        plot_bgcolor="#2E2E3E",   
                        font_color="#E0E0E0")
    return fig


In [16]:
def resumen_totales(df):
    total_carbs = df['carbs'].sum()
    total_protein = df['protein'].sum()
    total_fat = df['fat'].sum()
    total_kcal = df['calories_avg'].sum()

    return (
        f"### 🍽️ Resumen Nutricional (últimas 24h)\n"
        f"- 🍞 **Carbohidratos**: {total_carbs:.1f} g\n"
        f"- 🍗 **Proteína**: {total_protein:.1f} g\n"
        f"- 🥑 **Grasa**: {total_fat:.1f} g\n"
        f"- 🔥 **Calorías Totales**: {total_kcal:.1f} kcal"
    )


In [17]:
def cargar_historial(db_path="Nutrivision_log.db"):
    macro_color_map = {
        'carbs': '#FF9999',
        'protein': '#99CC99', 
        'fat': '#9999FF'
    }
    
    df = load_records()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['calories_avg'] = df[['calories', 'calories_from_macros']].mean(axis = 1)
    ahora = datetime.now()
    ayer = ahora - timedelta(days = 1)
    df = df[df['timestamp'] >= ayer]
    
    df_long = df.melt(id_vars='timestamp', value_vars=['carbs', 'protein', 'fat'],
                      var_name='Macronutriente', value_name='Gramos')
    fig_macros = px.line(
    df_long,
    x='timestamp',
    y='Gramos',
    color='Macronutriente',
    color_discrete_map=macro_color_map,
    title="Macronutrientes vs Tiempo"
    )

    fig_macros.update_layout(
        yaxis_title="Cantidad (g)",
        xaxis_title="Tiempo",
        template="none",
        paper_bgcolor="#1E1E2E",
        plot_bgcolor="#2E2E3E",
        font_color="#E0E0E0",
        legend_title_text="Macronutriente"
    )
    
    
    fig_cal = px.line(
        df,
        x='timestamp',
        y='calories_avg',
        title="Calorías vs Tiempo",
        labels={'calories_avg': 'Calorías', 'timestamp': 'Tiempo'}
    )
    
    fig_cal.update_traces(line=dict(color="#FFD580"))  
    
    fig_cal.update_layout(
        yaxis_title="Calorías",
        xaxis_title="Tiempo",
        template="none",
        paper_bgcolor="#1E1E2E",
        plot_bgcolor="#2E2E3E",
        font_color="#E0E0E0"
    )

    df_display = df.copy()
    df_display = df_display.drop(columns = ['calories_from_macros','calories'])
    df_display[['carbs', 'protein', 'fat', 'calories_avg']] = df_display[[
    'carbs', 'protein', 'fat', 'calories_avg']].round(2)
    resumen_dia = resumen_totales(df)
    return df_display, fig_macros, fig_cal, resumen_dia


In [18]:
def guardar_registro(nutrition_json,confirmar,db_path="Nutrivision_log.db"):
    if nutrition_json is None:
        raise gr.Error("No se puede guardar porque no hay datos nutricionales.")
    try:
        tz = pytz.timezone("America/Mexico_City")
        timestamp_now = datetime.now(tz)
        timestamp_str = timestamp_now.strftime('%Y-%m-%d %H:%M:%S')
        conn = sqlite3.connect(db_path)
        c = conn.cursor()
        hace_5_min = (timestamp_now - timedelta(minutes = 5)).strftime('%Y-%m-%d %H:%M:%S')
        query = """
        SELECT COUNT(*) FROM registros 
        WHERE  timestamp >= ? AND carbs = ? AND protein = ? AND fat = ?
        AND calories = ? AND calories_from_macros = ?
        
        """
        valores = (
        hace_5_min,
        nutrition_json["carbs"],
        nutrition_json["protein"],
        nutrition_json["fat"],
        nutrition_json["calories"],
        nutrition_json["calories_from_macros"],
            
        )
        c.execute(query, valores)
        count = c.fetchone()[0]
        if count > 0 and not confirmar:
            conn.close()
            gr.Info("⚠️ Ya existe un registro similar recientemente. Presiona 'Guardar Registro' otra vez para confirmar.")
            return True
        
        c.execute("""
            INSERT INTO registros (timestamp,carbs, protein, fat, calories, calories_from_macros)
            VALUES (?, ?, ?, ?, ? ,?)
        """, (
            timestamp_str,
            nutrition_json["carbs"],
            nutrition_json["protein"],
            nutrition_json["fat"],
            nutrition_json["calories"],
            nutrition_json["calories_from_macros"]
        ))
        conn.commit()
        conn.close()
        gr.Info("✅ Registro guardado exitosamente.")
        return False
    except Exception as e:
        raise Exception

    

In [19]:
def procesar_img(imagen):
    if imagen is None:
        return "⚠️ Por favor, sube una imagen antes de continuar.", None, None, None, None, None, False
    
    buf = io.BytesIO()
    imagen.save(buf, format="JPEG")
    img_bytes = buf.getvalue()
    
    if not openai_is_food(img_bytes):
        return "❌ No parece una comida reconocible.", None, None, None, None, None, False

    nutrition = predict_nutrition(imagen)
    macros_cal = nutrition_summary(nutrition)
    recomendacion = recomendacion_LLM(nutrition)
    fig = macros_barplot(nutrition)
    fig_pie = calorie_pie(nutrition)
    return macros_cal, recomendacion, fig, fig_pie, nutrition

### Interfaz

In [20]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    init_db()
    confirmar_guardado = gr.State(False)        
    
    with gr.Tab('Analisis de Imagen'):
        with gr.Row():
            input_img = gr.Image(type="pil", label="Sube una imagen de tu comida desde arriba y dejando ver los ingredientes")
    
        with gr.Row():
            analizar_btn = gr.Button("🔍 Analizar")
            limpiar_btn = gr.Button("🧹 Limpiar")
            guardar_btn = gr.Button("💾 Guardar Registro")
    
        json_nut_output = gr.State()
    
        out_resume = gr.Markdown(label="Resultado nutricional")  
        
        with gr.Row():
            out_plot = gr.Plot(label="Grafico")
            out_recomendacion = gr.Markdown(label="Recomendaciones")    
        
        out_plot_pie = gr.Plot(label="Grafico Pastel")
        
        analizar_btn.click(fn=procesar_img, inputs=[input_img], outputs=[out_resume,out_recomendacion,out_plot, out_plot_pie, json_nut_output])
        limpiar_btn.click(fn=limpiar, inputs=None, outputs=[input_img, out_resume, out_recomendacion,out_plot, out_plot_pie, json_nut_output, confirmar_guardado])
        guardar_btn.click(fn = guardar_registro,inputs = [json_nut_output, confirmar_guardado], outputs = [confirmar_guardado])
    
    with gr.Tab("Historial y Tendencias") as tab_historial:
        with gr.Row():
            salida_tabla = gr.Dataframe(label="📋 Historial de comidas")
            salida_resumen_dia = gr.Markdown(label = 'Resumen del dia')
        
        graf_macro = gr.Plot(label="Macronutrientes vs Tiempo")
        graf_calorias = gr.Plot(label="Calorías vs Tiempo")

        tab_historial.select(
            fn=cargar_historial,
            inputs=[],
            outputs=[salida_tabla,graf_macro,graf_calorias, salida_resumen_dia]
        )
        
    

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a584fcf5e3c7b4fdad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/home/paul-h-roldan-hernandez/Diplo_env/lib/python3.12/site-packages/gradio/blocks.py:1816: UserWarning:

A function (procesar_img) returned too many output values (needed: 5, returned: 7). Ignoring extra values.
    Output components:
        [markdown, markdown, plot, plot, state]
    Output values returned:
        ["⚠️ Por favor, sube una imagen antes de continuar.", None, None, None, None, None, False]

I0000 00:00:1760059687.355227    6415 service.cc:152] XLA service 0x797cfc047cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760059687.355244    6415 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-10-09 19:28:07.420104: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1760059687.855580    6415 cuda_dnn.cc:529] Loaded cuDNN version 91002
2025-10-09